In [1]:
import tensorflow as tf

# load the model used for prediction the external contour
ex_cnt_model = tf.keras.models.load_model('unet_model')
in_cnt_model = tf.keras.models.load_model('internal_smaller_mask_unet_model')

In [2]:
# load the image
original_images_dir = 'data/original/'
heatmap_images_dir = 'data/heatmap/'

In [3]:
import numpy as np
import tensorflow as tf

def get_contour(mask: np.ndarray):
    # Apply thresholding
    ret, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    # Find contour using the eroded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Find the contour with the largest area
    contour = max(contours, key=cv2.contourArea)
    return contour

def predict_mask(img, model):
    # Predict the mask
    img = cv2.resize(img, (128, 128))
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=-1)
    img = img / 255
    pred = model.predict(img)
    pred = tf.argmax(pred, axis=-1)
    pred = pred[..., tf.newaxis]
    pred = tf.squeeze(pred)
    pred = pred.numpy()
    pred = pred.astype(np.uint8) * 255
    return pred

In [4]:
def get_heatmap(img, contour):
    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # create a mask with the contour
    mask = np.zeros(img_grey.shape, np.uint8)
    cv2.drawContours(mask, [contour], -1, (255, 255, 255), -1, cv2.LINE_AA)
    #######################
    out = np.zeros_like(img)
    out[mask == 255] = img[mask == 255]
    img_color = cv2.cvtColor(out, cv2.COLOR_BGR2RGB)
    ######################
    img_grey = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(img_grey, (13,13), 0)
    #edges = cv2.Canny(blur, 50, 150)
    # Run Sobel filter on the original image
    sobelx = cv2.Sobel(blur, cv2.CV_8U, 1, 0, ksize=5)
    sobely = cv2.Sobel(blur, cv2.CV_8U, 0, 1, ksize=5)
    # Find the magnitude of the gradient
    sobel_magnitude = np.sqrt(sobelx**2 + sobely**2)
    # Normalize the image
    sobel_magnitude = sobel_magnitude/sobel_magnitude.max()*255
    # Convert to uint8
    sobel_magnitude = np.uint8(sobel_magnitude)

    window_size = 201 # Odd number only
    window_step = 10

    heat_map_sobel = np.zeros(sobel_magnitude.shape, dtype=np.uint8)

    # pad the images with zeros
    sobel_c = np.pad(sobel_magnitude, int((window_size-1)/2), mode='constant', constant_values=0)
    mask_metal_c = np.pad(mask, int((window_size-1)/2), mode='constant', constant_values=0)

    for y in range(0, sobel_c.shape[0], window_step):
        for x in range(0, sobel_c.shape[1], window_step):
            window = sobel_c[y:y+window_size, x:x+window_size]
            mask_metal_window = mask_metal_c[y:y+window_size, x:x+window_size]/255
            if mask_metal_window.sum() == 0 or mask_metal_window[(int((window_size - 1) / 2)), (int((window_size - 1) / 2))] == 0:
                heat_map_sobel[y:y+window_size, x:x+window_size] = 0
            else:
                heat_map_sobel[y:y+window_step, x:x+window_step] = np.sum(window)/mask_metal_window.sum()

    heat_map_sobel = cv2.equalizeHist(heat_map_sobel)

    # # apply a threshold to remove low-intensity values
    # threshold = 50
    # heat_map_sobel[heat_map_sobel < threshold] = 0

    # Save the image with color map
    heat_map_sobel_tmp = heat_map_sobel.copy()
    heat_map_color_sobel = cv2.applyColorMap(heat_map_sobel_tmp, cv2.COLORMAP_JET)
    heat_map_color_sobel_rgb = cv2.applyColorMap(heat_map_color_sobel, cv2.COLOR_BGR2RGB)
    return heat_map_color_sobel


In [5]:
def crop_image(img, contour):
    # Create a mask with the contour
    mask = np.zeros(img.shape, np.uint8)
    cv2.drawContours(mask, [contour], -1, (255, 255, 255), -1, cv2.LINE_AA)
    # Apply the mask to the image
    img = cv2.bitwise_and(img, mask)
    return img

In [6]:
import cv2
import numpy as np
from shapely.geometry import Polygon

def read_image(file_path):
    return cv2.imread(file_path)

def find_center_of_mass(contour, color_img):
    # Find the center of mass of the contour
    hull = cv2.convexHull(contour)
    ellipse = cv2.fitEllipse(hull)
    (x, y), (MA, ma), angle = ellipse
    return (int(x), int(y))

def find_center_of_mass2(contour):
    # Find the center of mass of the contour
    hull = cv2.convexHull(contour)
    M = cv2.minEnclosingCircle(hull)
    return (int(M[0][0]), int(M[0][1]))

def calculate_vector(center1, center2):
    return (center2[0] - center1[0], center2[1] - center1[1])

def is_point_near_line(point, line_start, line_end, eps1):
    # Convert tuples to numpy arrays
    point = np.array(point)
    line_start = np.array(line_start)
    line_end = np.array(line_end)
    eps = eps1

    # Calculate the directional vector of the line
    line_vec = line_end - line_start

    # Calculate the vector from the start of the line to the point
    point_vec = point - line_start

    # Calculate the cross product and the norm
    cross_prod = np.cross(line_vec, point_vec)
    norm_line_vec = np.linalg.norm(line_vec)

    # Calculate the distance from the point to the line
    distance = np.linalg.norm(cross_prod) / norm_line_vec

    # Check if the distance is within the threshold
    if abs(distance) <= eps:
        return True
    else:
        return False

def find_points_near_line(points, line_start, line_end, eps1):
    step=0.1
    max_eps=10
    # Initialize an empty list to store points near the line
    points_near_line = []

    # Temporary variable for the current eps1 value
    current_eps = eps1

    # Loop until at least one point is found or max_eps is reached
    while len(points_near_line) < 1000  and current_eps <= max_eps:
        # Iterate through each point in the list
        for point in points:
            # Check if the point is near the line
            if is_point_near_line(point, line_start, line_end, current_eps):
                # If the point is near the line, add it to the list
                points_near_line.append(point)

        # Increase current_eps for the next iteration
        current_eps += step

    # Return the list of points near the line as a numpy array
    return np.array(points_near_line)

def is_point_on_left_or_right_side(point, line_start, line_end):
    # Convert tuples to numpy arrays
    point = np.array(point)
    line_start = np.array(line_start)
    line_end = np.array(line_end)

    # Calculate the vector representing the original line
    original_line_vector = line_end - line_start

    # Calculate the orthogonal line vector (rotate original line by 90 degrees)
    orthogonal_line_vector = np.array([-original_line_vector[1], original_line_vector[0]])

    # Define the end point of the new line, extending the orthogonal line from line_start
    new_line_end = line_start + orthogonal_line_vector

    # Calculate the vector from the start of the new line to the point
    point_vector = point - line_start

    # Calculate the vector representing the new line
    new_line_vector = new_line_end - line_start

    # Calculate the cross product (in 2D, it's a scalar)
    cross_prod = np.cross(new_line_vector, point_vector)

    # Determine the side based on the sign of the cross product
    if cross_prod > 0:
        return 'left'
    else:
        return 'right'

def find_farthest_point(contour, center, axis_angle):
    """Find the farthest point on the contour from the center along a specified axis."""
    farthest_distance = 0
    for point in contour:
        # Project each point onto the specified axis
        vector = np.array(point[0]) - np.array(center)
        axis_vector = np.array([np.cos(np.radians(axis_angle)), np.sin(np.radians(axis_angle))])
        distance = np.dot(vector, axis_vector)
        farthest_distance = max(farthest_distance, abs(distance))
    return farthest_distance

def adjust_ellipse(contour, new_center, vector):
    new_center = new_center[0]
    # Fit initial ellipse
    ellipse = cv2.fitEllipse(contour)
    (x, y), (MA, ma), angle = ellipse
    angle = np.degrees(np.arctan2(vector[1], vector[0]), dtype=np.float32)

    # Calculate the farthest point on the contour from the center along the specified axis
    farthest_distance = find_farthest_point(contour, new_center, angle)

    # Calculate new major and minor axes lengths
    new_major_axis = ma * (ma / MA)
    new_minor_axis = farthest_distance  * 2

    # Update the ellipse with the new parameters
    ellipse = (new_center, (new_minor_axis, new_major_axis), angle)

    return ellipse, farthest_distance

def find_ellipse(in_cnt, ex_cnt):
    ex_cnt_center = find_center_of_mass2(ex_cnt)
    in_cnt_center = find_center_of_mass(in_cnt, cv2.resize(heatmap_img, (128, 128)))

    vector = calculate_vector(ex_cnt_center, in_cnt_center)

    # Find points near the line
    eps1 = 0.1
    points_near_line = find_points_near_line(ex_cnt, ex_cnt_center, in_cnt_center, eps1)

    # Find points on the left and right side of the line
    left_points = []
    right_points = []
    for point in points_near_line:
        if is_point_on_left_or_right_side(point, ex_cnt_center, in_cnt_center) == 'left':
            left_points.append(point)
        else:
            right_points.append(point)

    left_points = np.array(left_points)
    right_points = np.array(right_points)

    # Check if the center of mass of the internal contour is on the left or right side of the line
    side = is_point_on_left_or_right_side(in_cnt_center, ex_cnt_center, in_cnt_center)
    if side == 'left':
        points = left_points
    else:
        points = right_points

    sample_point = points[0]

    ellipse, farthest_distance = adjust_ellipse(in_cnt, sample_point, vector)

    return ellipse, farthest_distance

def linear_interpolation(contour):
    new_contour = []
    for i in range(len(contour)):
        p1 = contour[i]
        p2 = contour[(i + 1) % len(contour)]  # Circular connection
        mid_point = (p1 + p2) / 2
        new_contour.extend([p1, mid_point])
    return np.array(new_contour)

In [24]:
import os
directories = ['test', 'train']

# Create an pdf file to save the results
from fpdf import FPDF
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)

i = 0
for directory in directories:
    for img_name in os.listdir(original_images_dir + directory):
        original_img = read_image(original_images_dir + directory + '/' + img_name)
        heatmap_img = read_image(heatmap_images_dir + directory + '/' + img_name)
        ex_cnt_mask = predict_mask(original_img, ex_cnt_model)
        in_cnt_mask = predict_mask(heatmap_img, in_cnt_model)
        int_mask = cv2.GaussianBlur(in_cnt_mask, (7, 7), 0)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
        int_mask = cv2.dilate(int_mask, kernel, iterations=1)
        ex_cnt = get_contour(ex_cnt_mask)
        in_cnt = get_contour(in_cnt_mask)
        ex_cnt = ex_cnt.astype(np.uint8)
        in_cnt = in_cnt.astype(np.uint8)
        factor = int(original_img.shape[0] / 128)
        ex_cnt = linear_interpolation(ex_cnt)
        in_cnt = linear_interpolation(in_cnt)
        ex_cnt = ex_cnt.astype(np.int32)
        in_cnt = in_cnt.astype(np.int32)
        ellipse, farthest_distance = find_ellipse(in_cnt, ex_cnt)
        ellipse = (tuple(map(int, ellipse[0])), tuple(map(int, ellipse[1])), ellipse[2])
        scaled_ellipse = ((ellipse[0][0] * factor, ellipse[0][1] * factor), (ellipse[1][0] * factor, ellipse[1][1] * factor), ellipse[2])
        ex_cnt = ex_cnt * factor
        in_cnt = in_cnt * factor
        # Drawing the ellipse
        ellipse_img = heatmap_img.copy()
        cv2.ellipse(ellipse_img, scaled_ellipse, (0, 255, 0), 50)

        color_line = (255, 255, 255)  # Dodger Blue, for visibility
        thickness_line = 30  # Slightly thinner for elegance
        # Improved text annotation
        font_face = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 15
        font_thickness = 50
        color_text = (255, 255, 255)  # Cyan for contrast with red and visibility

        # Find the radius of the circle of the external contour
        radius = cv2.minEnclosingCircle(ex_cnt)[1]
        
        # Draw the vector on the image of the minor axis with the letter 'a'
        cv2.line(
                    ellipse_img,
                    (int(scaled_ellipse[0][0]), int(scaled_ellipse[0][1])),
                    (int(scaled_ellipse[0][0] + scaled_ellipse[1][1] / 2 * np.cos(np.radians(scaled_ellipse[2] + 270))),
                     int(scaled_ellipse[0][1] + scaled_ellipse[1][1] / 2 * np.sin(np.radians(scaled_ellipse[2] + 270)))),
                    color_line,
                    thickness_line
                )
        text_position = (int(scaled_ellipse[0][0] + scaled_ellipse[1][1] / 2 * np.cos(np.radians(scaled_ellipse[2] + 270))),
                         int(scaled_ellipse[0][1] + scaled_ellipse[1][1] / 2 * np.sin(np.radians(scaled_ellipse[2] + 270))))
        cv2.putText(ellipse_img, 'b', text_position, font_face, font_scale, color_text, font_thickness)
        # mark in point the end of the line
        cv2.circle(ellipse_img, (int(scaled_ellipse[0][0] + scaled_ellipse[1][1] / 2 * np.cos(np.radians(scaled_ellipse[2] + 270))),
                                int(scaled_ellipse[0][1] + scaled_ellipse[1][1] / 2 * np.sin(np.radians(scaled_ellipse[2] + 270)))), 50, (255, 255, 255), -1)
        # Draw the vector on the image of the major axis with the letter 'b'
        cv2.line(
            ellipse_img,
            (int(scaled_ellipse[0][0]), int(scaled_ellipse[0][1])),
            (int(scaled_ellipse[0][0] + scaled_ellipse[1][0] / 2 * np.cos(np.radians(scaled_ellipse[2] + 180))),
             int(scaled_ellipse[0][1] + scaled_ellipse[1][0] / 2 * np.sin(np.radians(scaled_ellipse[2] + 180)))),
            color_line,
            thickness_line)
        text_position = (int(scaled_ellipse[0][0] + scaled_ellipse[1][0] / 2 * np.cos(np.radians(scaled_ellipse[2] + 180))),
                         int(scaled_ellipse[0][1] + scaled_ellipse[1][0] / 2 * np.sin(np.radians(scaled_ellipse[2] + 180))))
        # do so i can see the annotation
        cv2.putText(ellipse_img, 'a', text_position, font_face, font_scale, color_text, font_thickness)
        # mark in point the end of the line
        cv2.circle(ellipse_img, (int(scaled_ellipse[0][0] + scaled_ellipse[1][0] / 2 * np.cos(np.radians(scaled_ellipse[2] + 180))),
                                int(scaled_ellipse[0][1] + scaled_ellipse[1][0] / 2 * np.sin(np.radians(scaled_ellipse[2] + 180)))), 50, (255, 255, 255), -1)
        
        
        radius = cv2.minEnclosingCircle(ex_cnt)[1]
        
        # Drawing the line in opposite direction of the major axis and length of radius * 2
        cv2.line(
            ellipse_img,
            (int(scaled_ellipse[0][0]), int(scaled_ellipse[0][1])),
            (int(scaled_ellipse[0][0] + radius * 2 * np.cos(np.radians(scaled_ellipse[2] + 180))),
             int(scaled_ellipse[0][1] + radius * 2 * np.sin(np.radians(scaled_ellipse[2] + 180))),
             ),
            color_line,
            thickness_line
        )
        
        cv2.circle(ellipse_img, (int(scaled_ellipse[0][0] + radius * 2 * np.cos(np.radians(scaled_ellipse[2] + 180))),
                                int(scaled_ellipse[0][1] + radius * 2 * np.sin(np.radians(scaled_ellipse[2] + 180)))), 50, (255, 255, 255), -1)
        
        text_position = (int(scaled_ellipse[0][0] + radius * 2 * np.cos(np.radians(scaled_ellipse[2] + 180))),
                         int(scaled_ellipse[0][1] + radius * 2 * np.sin(np.radians(scaled_ellipse[2] + 180))))
        
        
        cv2.putText(ellipse_img, 'D', text_position, font_face, font_scale, color_text, font_thickness)
        
        # Add a description to the image with the ellipse major and minor axes lengths outside image
        b = round(scaled_ellipse[1][1] / 2, 2)
        a = round(scaled_ellipse[1][0] / 2, 2)
        D = round(radius * 2, 2)
        text = 'Major axis (b): ' + str(round(scaled_ellipse[1][1] / 2, 2)) + ' pixels'
        text += '\nMinor axis (a): ' + str(round(scaled_ellipse[1][0] / 2, 2)) + ' pixels'
        text += '\nRadius (D): ' + str(round(radius * 2, 2)) + ' pixels'
        text += '\nFarthest distance (h): ' + str(round(farthest_distance * factor / 2.0, 2)) + ' pixels'

        # plt.show()
        pdf.cell(200, 10, txt=img_name, ln=1, align="C")
        # two lines for the text
        pdf.multi_cell(0, 10, text)
        # reduce the size of the image
        ellipse_img = cv2.resize(ellipse_img, (0, 0), fx=0.5, fy=0.5)
        # Save the image in tmp file
        cv2.imwrite('tmp' + img_name + '.png', ellipse_img)
        # Add the image to pdf file
        pdf.image('tmp' + img_name + '.png', x=50, y=60, w=100)
        # Add new page
        pdf.add_page()
        os.remove('tmp' + img_name + '.png')
    
# Save the pdf file
pdf.output("ellipses.pdf")

1/1 [==============================] - 0s 76ms/step


''

In [9]:
M = np.array([
    # i = 0
    [
        [1.095, 0.113, -0.896],  # j = 0, k = 0 to 2
        [-1.336, 1.824, 3.092],  # j = 1, k = 0 to 2
        [13.108, -21.709, -4.197],  # j = 2, k = 0 to 2
        [-43.689, 105.483, -13.255],  # j = 3, k = 0 to 2
        [134.868, -271.225, 51.548],  # j = 4, k = 0 to 2
        [-242.653, 387.470, -59.329],  # j = 5, k = 0 to 2
        [254.093, -290.024, 13.481],  # j = 6, k = 0 to 2
        [-108.196, 88.387, 10.854]  # j = 7, k = 0 to 2
    ],
    # i = 1
    [
        [-1.177, 0.271, 0.904],  # j = 0, k = 0 to 2
        [17.924, -11.649, 0.701],  # j = 1, k = 0 to 2
        [-137.252, 98.358, -32.641],  # j = 2, k = 0 to 2
        [545.816, -415.027, 204.104],  # j = 3, k = 0 to 2
        [-1223.334, 982.713, -568.407],  # j = 4, k = 0 to 2
        [1541.587, -1329.634, 857.543],  # j = 5, k = 0 to 2
        [-1006.656, 961.893, -657.659],  # j = 6, k = 0 to 2
        [264.206, -288.565, 191.570]  # j = 7, k = 0 to 2
    ],
    # i = 2
    [
        [0.725, -0.388, 0.008],  # j = 0, k = 0 to 2
        [-17.427, 10.074, -4.883],  # j = 1, k = 0 to 2
        [134.652, -80.088, 55.092],  # j = 2, k = 0 to 2
        [-551.902, 328.165, -305.079],  # j = 3, k = 0 to 2
        [1239.493, -772.921, 916.962],  # j = 4, k = 0 to 2
        [-1548.537, 1055.952, -1545.428],  # j = 5, k = 0 to 2
        [969.388, -784.581, 1372.595],  # j = 6, k = 0 to 2
        [-227.132, 245.798, -485.556]  # j = 7, k = 0 to 2
    ]
])

In [11]:
import os
directories = ['test', 'train']

# Create an CSV file to save the results of the area of the external & internal contour
import csv
with open('crack_growth_calc.csv', mode='w') as csv_file:
    fieldnames = ['img_name', 'ex_cnt_area', 'in_cnt_area', 'a', 'b', 'D', 'F1 at the middle of the crack front', 'F1 at the edge of the crack front']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    i = 0
    for directory in directories:
        for img_name in os.listdir(original_images_dir + directory):
            original_img = read_image(original_images_dir + directory + '/' + img_name)
            heatmap_img = read_image(heatmap_images_dir + directory + '/' + img_name)
            ex_cnt_mask = predict_mask(original_img, ex_cnt_model)
            in_cnt_mask = predict_mask(heatmap_img, in_cnt_model)
            int_mask = cv2.GaussianBlur(in_cnt_mask, (7, 7), 0)
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
            int_mask = cv2.dilate(int_mask, kernel, iterations=1)
            ex_cnt = get_contour(ex_cnt_mask)
            in_cnt = get_contour(in_cnt_mask)
            ex_cnt = ex_cnt.astype(np.uint8)
            in_cnt = in_cnt.astype(np.uint8)
            factor = int(original_img.shape[0] / 128)
            ex_cnt = linear_interpolation(ex_cnt)
            in_cnt = linear_interpolation(in_cnt)
            ex_cnt = ex_cnt.astype(np.int32)
            in_cnt = in_cnt.astype(np.int32)
            ellipse, farthest_distance = find_ellipse(in_cnt, ex_cnt)
            ellipse = (tuple(map(int, ellipse[0])), tuple(map(int, ellipse[1])), ellipse[2])
            scaled_ellipse = ((ellipse[0][0] * factor, ellipse[0][1] * factor), (ellipse[1][0] * factor, ellipse[1][1] * factor), ellipse[2])
            ex_cnt = ex_cnt * factor
            in_cnt = in_cnt * factor
            # Drawing the ellipse
            ellipse_img = heatmap_img.copy()
            cv2.ellipse(ellipse_img, scaled_ellipse, (0, 255, 0), 50)

            # Find the radius of the circle of the external contour
            radius = cv2.minEnclosingCircle(ex_cnt)[1]

            a = round(scaled_ellipse[1][0] / 2, 2)
            b = round(scaled_ellipse[1][1] / 2, 2)
            D = round(radius * 2, 2)

            K_c = 0
            F1 = 0
            sigma = 634

            for i in range(3):
                for j in range(8):
                    for k in range(3):
                        a_div_b = a / b
                        if a_div_b > 1:
                            a_div_b = 1
                        F1 += M[i][j][k] * ((a_div_b ** i) * ((a / D) ** j) * (0 ** k))
            F_c = F1

            K_m = F1 * sigma * np.sqrt(np.pi * a) *  ((5500/4096))
            K_e = 0
            F1 = 0
            sigma = 634

            for i in range(3):
                for j in range(8):
                    for k in range(3):
                        # create a ceil of up to 1 to a/b
                        a_div_b = a / b
                        if a_div_b > 1:
                            a_div_b = 1
                        F1 += M[i][j][k] * ((a_div_b ** i) * ((a / D) ** j) * (1 ** k))
            F_e = F1
            k_e = F1 * sigma * np.sqrt(np.pi * a) * ((5500/4096))

            # add a row with the area of the external & internal contour, a, b, D
            writer.writerow({'img_name': img_name, 'ex_cnt_area': round(cv2.contourArea(ex_cnt), 2), 'in_cnt_area': round(cv2.contourArea(in_cnt), 2), 'a': a, 'b': b, 'D': D, 'F1 at the middle of the crack front': F_c, 'F1 at the edge of the crack front': F_e})
            
            i += 1


1/1 [==============================] - 0s 48ms/step
